In [1]:
import pandas as pd
import statsmodels.api as sm
import numpy as np
import time
import pystan

C:\Users\natan\Anaconda3\lib\site-packages\statsmodels\compat\pandas.py:49: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  data_klasses = (pandas.Series, pandas.DataFrame, pandas.Panel)


In [361]:
df=pd.read_csv('./obama_tweets/with_financials/obama_twitter_data_with_vix_for_analysis.csv',parse_dates=['created_at'],index_col=0)

In [362]:
for x in df.columns:
    print(x)

text
Name
Screen Name
created_at
favorite_count
retweet_count
Language
Client
Tweet Type
Media Type
Media URLs
URLs
Hashtags
Mentions
trade_topic
china_topic
financial_topic
labor_topic
industry_topic
currency_rates_topic
general_related_topic
budget_topic
foreign_policy_topic
covid_topic
sensitive_topic
environment_topic
sentiment
month
1_minutes_forward_pct_change
5_minutes_forward_pct_change
10_minutes_forward_pct_change
15_minutes_forward_pct_change
30_minutes_forward_pct_change
1_hours_forward_pct_change
3_hours_forward_pct_change


In [363]:
X=df.loc[:,['created_at',
'favorite_count',
'retweet_count',
'trade_topic',
'china_topic',
'financial_topic',
'labor_topic',
'industry_topic',
'currency_rates_topic',
'general_related_topic',
'budget_topic',
'foreign_policy_topic',
'covid_topic',
'sensitive_topic',
'environment_topic',
'sentiment',
'text'
]].copy()

In [364]:
y=df[['1_minutes_forward_pct_change',
'5_minutes_forward_pct_change',
'10_minutes_forward_pct_change',
'15_minutes_forward_pct_change',
'30_minutes_forward_pct_change',
'1_hours_forward_pct_change']].copy()
# '1_minutes_forward_vol_sum',
# '5_minutes_forward_vol_sum',
# '10_minutes_forward_vol_sum',
# '15_minutes_forward_vol_sum',
# '30_minutes_forward_vol_sum',
# '1_hours_forward_vol_sum']].copy()

In [365]:
inspect_df=pd.concat([X.loc[~y['5_minutes_forward_pct_change'].isna()],y['5_minutes_forward_pct_change'].dropna()],axis=1)

In [366]:
inspect_df=inspect_df.loc[inspect_df['created_at']>='2016-11-08 22:00:00'].copy()

In [367]:
for column in inspect_df.iloc[:,:-1].columns:
    inspect_df[column] = inspect_df[[column, 'sentiment']].product(axis=1)
# inspect_df.drop('sentiment',1,inplace=True)

In [368]:
inspect_df.drop(['created_at','text'],1,inplace=True)

In [369]:
# inspect_df=inspect_df.replace([np.inf, -np.inf], float('nan'))
inspect_df=inspect_df.dropna()

In [370]:
mod = sm.OLS(inspect_df.iloc[:,-1],sm.add_constant(inspect_df.iloc[:,:-1]), hasconst = False)
res = mod.fit()

In [371]:
print(res.summary())

                                 OLS Regression Results                                 
Dep. Variable:     5_minutes_forward_pct_change   R-squared:                       0.043
Model:                                      OLS   Adj. R-squared:                  0.022
Method:                           Least Squares   F-statistic:                     2.073
Date:                          Mon, 13 Jul 2020   Prob (F-statistic):             0.0308
Time:                                  20:49:54   Log-Likelihood:                 1507.8
No. Observations:                           420   AIC:                            -2998.
Df Residuals:                               411   BIC:                            -2961.
Df Model:                                     9                                         
Covariance Type:                      nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
--------------------

# MCMC inspection

In [372]:
# Specify the model in Stan

model = """
data {
    int<lower=0> N;
    vector[N] x;
    vector[N] x1;
    vector[N] x2;
    vector[N] x3;
    vector[N] x4;
    vector[N] x5;
    vector[N] x6;
    vector[N] x7;
    vector[N] x8;
    vector[N] x9;
    vector[N] x10;
    vector[N] x11;
    vector[N] x12;
    vector[N] x13;
    vector[N] x14;
    vector[N] y;
}
parameters {
    real alpha;
    real favorite_count;
    real retweet_count;
    real trade_topic;
    real china_topic;
    real financial_topic;
    real labor_topic;
    real industry_topic;
    real currency_rates_topic;
    real general_related_topic;
    real budget_topic;
    real foreign_policy_topic;
    real covid_topic;
    real sensitive_topic;
    real environment_topic;
    real sentiment;
    real<lower=0> sigma;
}
model {
    y ~ normal(alpha + favorite_count * x + retweet_count * x1 + trade_topic * x2 + china_topic * x3 + financial_topic * x4 + labor_topic * x5 + industry_topic * x6 + currency_rates_topic * x7 + general_related_topic * x8 + budget_topic * x9 + foreign_policy_topic * x10 + covid_topic * x11 + sensitive_topic * x12 + environment_topic * x13 + sentiment * x14, sigma);
}
"""

In [373]:
# implement the model using Pystan

data = {'N': len(inspect_df), 'x': inspect_df.iloc[:,0], 'x1': inspect_df.iloc[:,1], 'x2': inspect_df.iloc[:,2], 
        'x3': inspect_df.iloc[:,3], 'x4': inspect_df.iloc[:,4], 'x5': inspect_df.iloc[:,5], 'x6': inspect_df.iloc[:,6], 
        'x7': inspect_df.iloc[:,7], 'x8': inspect_df.iloc[:,8], 'x9': inspect_df.iloc[:,9], 'x10': inspect_df.iloc[:,10], 
        'x11': inspect_df.iloc[:,11], 'x12': inspect_df.iloc[:,12], 'x13': inspect_df.iloc[:,13], 
        'x14': inspect_df.iloc[:,14], 'y': inspect_df.iloc[:,-1]}

# Compile the model
smod = pystan.StanModel(model_code=model)

# Train the model and generate samples
fit = smod.sampling(data=data, iter=1000, chains=4, warmup=500, thin=1, seed=101)

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_4cd63eaaa8fafd9a7c8d59c85037c311 NOW.


In [374]:
summary_dict = fit.summary()
df_summary = pd.DataFrame(summary_dict['summary'], 
                  columns=summary_dict['summary_colnames'], 
                  index=summary_dict['summary_rownames'])

In [375]:
# Pystan results
df_summary

,mean,se_mean,sd,2.5%,25%,50%,75%,97.5%,n_eff,Rhat
alpha,3.147765e-04,8.651243e-06,3.687477e-04,-3.869746e-04,5.836397e-05,3.238009e-04,5.576776e-04,1.046671e-03,1816.777585,1.001493
favorite_count,-1.292156e-03,4.886797e-05,2.169283e-03,-5.652972e-03,-2.741826e-03,-1.219406e-03,1.205671e-04,2.974458e-03,1970.533596,0.998851
retweet_count,-8.852199e-04,4.713894e-05,2.058230e-03,-4.924449e-03,-2.193147e-03,-9.688618e-04,4.391876e-04,3.240689e-03,1906.462863,0.998667
trade_topic,3.783715e+06,7.262230e+07,1.088163e+08,-2.013532e+08,-4.113772e+07,3.005249e+06,8.943294e+07,1.597087e+08,2.245164,3.414016
china_topic,-4.870512e+05,3.004377e+07,9.792441e+07,-2.048962e+08,-5.336268e+07,-1.455525e+06,2.761756e+07,2.254358e+08,10.623635,1.545613
financial_topic,2.549471e-04,1.091877e-04,4.918649e-03,-9.194514e-03,-3.191582e-03,2.335824e-04,3.720835e-03,9.708289e-03,2029.289243,1.000558
labor_topic,5.291086e-03,4.595561e-05,2.206918e-03,1.000582e-03,3.675516e-03,5.305789e-03,6.753526e-03,9.589214e-03,2306.191831,0.999740
industry_topic,-1.663326e+07,1.171011e+07,2.025015e+07,-5.534614e+07,-3.197881e+07,-1.970408e+07,1.077538e+06,1.620300e+07,2.990433,2.426920
currency_rates_topic,-2.199935e+07,2.933999e+07,4.700008e+07,-1.029957e+08,-6.240419e+07,-1.346337e+07,1.082715e+07,5.689086e+07,2.566122,3.426279
general_related_topic,-3.857979e-03,1.627317e-04,6.894586e-03,-1.659549e-02,-8.765213e-03,-4.090735e-03,8.265994e-04,9.855780e-03,1795.032618,0.999248


In [376]:
df_summary.to_csv('mcmc_obama_vix_price_after_2016.csv')

In [377]:
probs = pd.DataFrame()
columns = ['favorite_count', 'retweet_count', 'trade_topic', 'china_topic',
       'financial_topic', 'labor_topic', 'industry_topic',
       'currency_rates_topic', 'general_related_topic', 'budget_topic',
       'foreign_policy_topic', 'covid_topic', 'sensitive_topic',
       'environment_topic', 'sentiment']
for column in columns:
    probs[column] = fit[column]

In [378]:
probs.to_csv('extended_mcmc_obama_vix_price_after_2016.csv')

# Generalization

In [ ]:
from sklearn.linear_model import Ridge
rr = Ridge(alpha=0.00000000001,normalize=False)
rr.fit(inspect_df.iloc[:,:-1], inspect_df.iloc[:,-1])
w = rr.coef_

In [ ]:
print(rr.score(inspect_df.iloc[:,:-1], inspect_df.iloc[:,-1]))

In [ ]:
from sklearn import linear_model
clf = linear_model.Lasso(alpha=0.000000001,normalize=False)
clf.fit(inspect_df.iloc[:,:-1], inspect_df.iloc[:,-1])
print(clf.score(inspect_df.iloc[:,:-1], inspect_df.iloc[:,-1]))

In [ ]:
coefs=pd.DataFrame(columns=['variable','coefficients'])

In [ ]:
coefs['variable']=inspect_df.iloc[:,:-1].columns

In [ ]:
coefs['coefficients'] = clf.coef_

In [ ]:
len(coefs[coefs['coefficients']!=0])

In [ ]:
for i,x in coefs[coefs['coefficients']!=0].sort_values('coefficients').iterrows():
    print(x['variable'])